In [18]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re
import sys
import time

import numpy as np
import pandas as pd
import os
import psycopg2

from tqdm import tqdm

In [19]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [20]:
skiplist

[115922,
 119509,
 119510,
 18780,
 25273,
 28631,
 29112,
 31395,
 32583,
 33032,
 33276,
 115922,
 119509,
 119510,
 25273,
 115922,
 119509,
 119510,
 25273,
 67583,
 68459,
 80854,
 100,
 1000,
 100000,
 100024,
 10008,
 100082,
 100086,
 100088,
 100092,
 100094,
 100098,
 100100,
 100122,
 100143,
 100151,
 100153,
 100160,
 116398,
 10018,
 100186,
 100192,
 100195,
 100209,
 100221,
 100222,
 100230,
 100236,
 104692,
 100245,
 100247,
 10026,
 100263,
 100271,
 100274,
 100279,
 100281,
 100297,
 10031,
 100333,
 100334,
 100337,
 10035,
 100358,
 10036,
 100367,
 100369,
 10037,
 100371,
 100387,
 100388,
 100390,
 100418,
 101909,
 102057,
 102058,
 102059,
 10206,
 102060,
 102061,
 102062,
 102063,
 102064,
 102065,
 102066,
 102067,
 102068,
 102069,
 102070,
 102071,
 102072,
 102073,
 102074,
 102075,
 102076,
 102077,
 102078,
 102079,
 10208,
 102080,
 102081,
 102082,
 102083,
 102084,
 102085,
 102086,
 102087,
 102088,
 102089,
 10209,
 102090,
 102091,
 102092,
 1

In [21]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

In [22]:
command = """
SELECT doc_id,doc_url
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 5
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [23]:
existing

[(10456,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/fz_677316/1206_678356/1206x2_678376/t1550313.shtml'),
 (104560,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681266/xgxw_681272/t457467.shtml'),
 (104561,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681266/xgxw_681272/t359767.shtml'),
 (104562,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681266/xgxw_681272/t276120.shtml'),
 (104563,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681266/xgxw_681272/t275962.shtml')]

In [24]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT count(doc_id)
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}

""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [25]:
existing

[(107996,)]

In [26]:
command = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'documents'
"""

cur.execute(command)

columns = cur.fetchall()

In [27]:
columns

[('doc_id',),
 ('doc_country',),
 ('doc_language',),
 ('doc_page_name',),
 ('doc_url',),
 ('doc_nav',),
 ('doc_title',),
 ('doc_date',),
 ('doc_html',),
 ('doc_content',),
 ('doc_skip',)]

In [28]:
def html(url):
    try:
        driver.get(url)
        html = driver.page_source
        if type(html) == str:
            return(html)
        else:
            return('ERROR')
    except:
        return('ERROR')

In [29]:
def headlessDriver():
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="geckodriver.exe")
    return(driver)

In [30]:
def buildEntry(index_value,url):
    t_start = time.time()
    
    index = index_value
    pageSource = html(url)
#     pageDate = str()
    pageText = str()
    if pageSource == 'ERROR':
#         pageDate = 'ERROR'
        pageText = 'ERROR'
    else:
        try:
            soup=BeautifulSoup(pageSource, 'lxml')
#             pageDate = soup.find("span", {"id": "News_Body_Time"}).text
#             if pageDate == '':
#                 pageDate = None
            pageText = soup.find(class_='content').text
            if pageText == '':
                pageText = None
        except:
#             pageDate = 'ERROR'
            pageText = 'ERROR'
    
    entry = (pageSource,
#              pageDate,
             pageText,
             index)
    
    t_end = time.time()
    
    duration = t_end - t_start
    
    if duration >= 10:
        time.sleep(np.random.randint(5,15))
    
#   print(duration)          
    return(entry)

In [31]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT doc_id,doc_url
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 10
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

cur.close()
conn.close()

In [32]:
def fullHarvest():
    t = time.time()
    try:
        skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
    except:
        skiplist = []
        
    
    limit = 1
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    
    command = """
    SELECT doc_id,doc_url
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    LIMIT {}
    """.format(tuple(skiplist),limit)

    cur.execute(command)

    existing = cur.fetchall()
#     print(existing)
    cur.close()
    conn.close()
    
#     driver = None
#     driver = headlessDriver()
    submission = [buildEntry(int(x[0]),str(x[1])) for x in existing]
    
    newSubmission = []
    for i in submission:
        if i[1] == 'ERROR':
            skiplist.append(i[2])
        else:
            newSubmission.append(i)
    
    
    pd.Series(skiplist).to_csv('skiplist.csv')
    
    
    sub_len = len(newSubmission)
    
    
    
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    sql = """
        UPDATE documents
        SET doc_html = %s,doc_content = %s
        WHERE doc_id = %s;
        """

    cur.executemany(sql,newSubmission)
    conn.commit()
    
    
    command = """
    SELECT COUNT(doc_id)
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    """.format(tuple(skiplist))


    cur.execute(command)

    count = cur.fetchall()
    

    cur.close()
    conn.close()
    
    nt = time.time()
    dt = nt - t
    
    
    
    return(len(skiplist),count,dt,sub_len,existing)

    

In [33]:
counter = 0
driver = headlessDriver()

for i in tqdm(range(30000)):    
    s,r,t,l,e = fullHarvest()
    print("{} : {:.4f}s : {} skips : {} remaining : {} counter".format(i,t,s,r,counter))
    if l == 0:
        print(e)
        counter += 1
        if counter > 10:
            break
        else:
            time.sleep(np.random.randint(15,30))
            driver = headlessDriver()
            
        



  0%|          | 0/30000 [00:00<?, ?it/s]

0 : 11.3732s : 883 skips : [(107995,)] remaining : 0 counter




  0%|          | 1/30000 [00:11<94:47:55, 11.38s/it]

1 : 9.0455s : 883 skips : [(107994,)] remaining : 0 counter




  0%|          | 2/30000 [00:20<88:59:02, 10.68s/it]

...

4794 : 4.3656s : 887 skips : [(103201,)] remaining : 4 counter




 16%|█▌        | 4795/30000 [8:49:44<30:47:48,  4.40s/it]

4795 : 4.3606s : 887 skips : [(103200,)] remaining : 4 counter




 16%|█▌        | 4796/30000 [8:49:49<30:43:26,  4.39s/it]

KeyboardInterrupt: 

In [ ]:
# buildEntry(104328,"https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/1206x2_681262/t1145802.shtml")

In [ ]:
# driver = headlessDriver()
# buildEntry(102358,"""https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681192/1206x2_681212/t878484.shtml""")

In [ ]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [ ]:
len(skiplist)

In [ ]:
skiplist

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()
# command = """
# ALTER TABLE documents 
# ADD COLUMN doc_skip SMALLINT
# """
# cur.execute(command)
# conn.commit()

# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT column_name
# FROM information_schema.columns
# WHERE table_name = 'documents'
# """

# cur.execute(command)

# columns = cur.fetchall()


# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# columns

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# sql = """
#         UPDATE documents
#         SET doc_skip = 1
#         WHERE doc_id IN {};
#         """.format(tuple(skiplist))

# cur.execute(sql)
# conn.commit()



# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# sql = """
#         SELECT * 
#         FROM documents
#         WHERE doc_skip = 1
#         LIMIT 10
#         """

# cur.execute(sql)

# sample = cur.fetchall()

# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# sample

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT doc_id,doc_url
# FROM documents
# WHERE doc_url NOT SIMILAR TO '%shtml' AND doc_id NOT IN {}
# """.format(tuple(skiplist))


# cur.execute(command)

# extract = cur.fetchall()
# print(extract)
# cur.close()
# conn.close()

In [ ]:
# submission[-1]

In [ ]:
# for i in submission:
#     print(i[2],'\n')

In [ ]:
# print(len(submission))

In [ ]:
# for i in submission:
#     if i[2] == 'ERROR':
#         print(i[3])

In [ ]:
# newSubmission = []
# for i in submission:
#     if i[1] == 'ERROR':
#         skiplist.append(i[3])
#     else:
#         newSubmission.append(i)

In [ ]:
# pd.Series(skiplist).to_csv('skiplist.csv')

In [ ]:
# pd.Series(skiplist)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

In [ ]:
# sql = """
#         UPDATE documents
#         SET doc_html = %s,doc_date = %s,doc_content = %s
#         WHERE doc_id = %s;
#         """

# cur.executemany(sql,newSubmission)
# conn.commit()

# cur.close()
# conn.close()

In [ ]:
# existing

In [ ]:
# len(existing)

In [ ]:
# listCheck = [x[0] for x in existing]

In [ ]:
# listCheck

In [ ]:
# tuple(listCheck)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT *
# FROM documents
# WHERE doc_id IN {}
# """.format(tuple(listCheck))

# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT COUNT(doc_id)
# FROM documents
# WHERE doc_html IS NULL AND doc_date IS NULL and doc_id NOT IN {}
# """.format(tuple(skiplist))


# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# extract